In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
import os

os.environ['HADOOP_USER_NAME'] = 'root'

# 1. Создаем сессию Spark
# Имя хоста 'namenode' берем из нашего docker-compose
spark = SparkSession.builder \
    .appName("QuickHDFSTest") \
    .master("local[*]") \
    .config("dfs.client.use.datanode.hostname", "true") \
    .getOrCreate()

# 2. Тестовые данные
data = [("Ivan", 25), ("Anna", 30), ("Petr", 18), ("Elena", 45)]

# 3. Описываем схему
schema = StructType([
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True)
])

# 4. Создаем DataFrame
df = spark.createDataFrame(data, schema)

# 5. Показываем результат в консоли
print("Наш легкий DataFrame:")
df.show()

# 6. Записываем в HDFS (путь внутри контейнера к NameNode)
# Мы создадим папку /user/spark_test
try:
    df.write.mode("overwrite").parquet("hdfs://namenode:9000/user/spark_test")
    print("✅ Успешно сохранено в HDFS!")
except Exception as e:
    print(f"❌ Ошибка записи: {e}")

# 7. Читаем обратно для проверки
print("Чтение записанных данных из HDFS:")
df_load = spark.read.parquet("hdfs://namenode:9000/user/spark_test")
df_load.filter(df_load.age > 20).show()

spark.stop()


Наш легкий DataFrame:
+-----+---+
| name|age|
+-----+---+
| Ivan| 25|
| Anna| 30|
| Petr| 18|
|Elena| 45|
+-----+---+

✅ Успешно сохранено в HDFS!
Чтение записанных данных из HDFS:
+-----+---+
| name|age|
+-----+---+
|Elena| 45|
| Ivan| 25|
| Anna| 30|
+-----+---+



In [6]:
data = [(1, "Gold"), (2, "Silver")]
df = spark.createDataFrame(data, ["client_id", "segment"])

# Сохраняем как таблицу (Spark создаст метаданные в папке spark-warehouse)
df.write.mode("overwrite").saveAsTable("clients")

# Теперь SQL запрос будет работать!
spark.sql("SELECT * FROM clients").show()

AttributeError: 'NoneType' object has no attribute 'sc'

In [12]:
spark.sql('''
CREATE TABLE default.clients (
    client_id INTEGER,
    segment STRING
) USING PARQUET;
''').show()


AnalysisException: Table default.clients already exists.

In [13]:
# spark.sql('''
# INSERT INTO default.clients (client_id, segment) VALUES
# (1, 'retail'),
# (2, 'retail'),
# (3, 'premium'),
# (4, 'corporate'),
# (5, 'premium')
# ''')

spark.sql("SELECT * FROM default.clients").show()


Py4JJavaError: An error occurred while calling o85.showString.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:77)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:499)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:480)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.comman